In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

import validphys
from validphys.commondataparser import parse_commondata_new, parse_commondata
from validphys.loader import _get_nnpdf_profile

## Old-New comparison

The following cells compare the output the old `parse_commondata` and the new `parse_commondata_new` (both from `commondataparser.py`).

The output of these two functions should be equal, since they parse the old/new commondata files into the `CommonData` class from `coredata.py`.

Note that for this comparison, my version of "NMCPD" is called "NMCPDF_fake" so that the old and new dataset don't clash.

In [2]:
#  for NMCPD
dataset = "E605_DY_38P8GEV_PXSEC"

old_name = "DYE605"


# Again, this assumes validphys installed in editable mode etc etc and we look at the defaults
old_data_path = Path(_get_nnpdf_profile()["data_path"]) / "commondata" / f"DATA_{old_name}.dat"
old_err_path = old_data_path.parent / "systypes" / f"SYSTYPE_{old_name}_DEFAULT.dat"
old_cd = parse_commondata(old_data_path, old_err_path, old_name)

# If this trick doesn't find your data, put here the path to wherever your data is
folder_data = Path(validphys.__file__).parent / "../../../buildmaster"

new_cd = parse_commondata_new(dataset)

In [3]:
old_cd.commondata_table

,process,kin1,kin2,kin3,data,stat,ADD,MULT,ADD,MULT
entry,,,,,,,,,,
1,DYP_E605,-0.2,50.470794,38.8,364.000,106.000,36.4000,10.0,54.60000,15.0
2,DYP_E605,-0.2,53.378220,38.8,209.000,46.000,20.9000,10.0,31.35000,15.0
3,DYP_E605,-0.2,56.367061,38.8,220.000,40.000,22.0000,10.0,33.00000,15.0
4,DYP_E605,-0.2,59.377504,38.8,243.000,46.000,24.3000,10.0,36.45000,15.0
5,DYP_E605,-0.2,62.527607,38.8,119.000,22.000,11.9000,10.0,17.85000,15.0
...,...,...,...,...,...,...,...,...,...,...
115,DYP_E605,0.4,127.920624,38.8,24.700,3.000,2.4700,10.0,3.70500,15.0
116,DYP_E605,0.4,146.639506,38.8,14.600,1.600,1.4600,10.0,2.19000,15.0
117,DYP_E605,0.4,177.216799,38.8,6.140,0.710,0.6140,10.0,0.92100,15.0


In [4]:
new_cd.commondata_table

,process,kin1,kin2,kin3,data,stat,MULT,ADD
entry,,,,,,,,
1,DY,1505.44,0.1831,-0.2,364.000,106.000,54.60000,18.20000
2,DY,1505.44,0.1883,-0.2,209.000,45.900,31.35000,10.45000
3,DY,1505.44,0.1935,-0.2,220.000,40.300,33.00000,11.00000
4,DY,1505.44,0.1986,-0.2,243.000,45.500,36.45000,12.15000
5,DY,1505.44,0.2038,-0.2,119.000,21.500,17.85000,5.95000
...,...,...,...,...,...,...,...,...
115,DY,1505.44,0.2915,0.4,24.700,3.030,3.70500,1.23500
116,DY,1505.44,0.3121,0.4,14.600,1.550,2.19000,0.73000
117,DY,1505.44,0.3431,0.4,6.140,0.709,0.92100,0.30700


## The new commondata

The following cells basically do what `parse_commondata_folder` does internally (wihtout creating the final `CommonData` object).

The first step is to read the metadata from `<datafolder>/metadata.yaml`. Most of the new logic is concentrated in this step since it uses `validobj` to ensure that all the information inside the `metadata` file is correct (very importantly, it checks the theory information which is also used by the pineappl parser later on!)

After the metadata has been read, the data, uncertainties, and kinematics are read as well into beautiful dataframes. The `new_cd.commondata_table` above is made from a simplified concatenation of said dataframes.

In [5]:
from validphys.utils import parse_yaml_inp
from validphys.commondataparser import Set, _parse_data, _parse_uncertainties, _parse_kinematics

commondata_folder = folder_data / "NMCPD_fake"
metadata_file = commondata_folder / "metadata.yaml"

# Select which variants you want
variants = []

# Read up all the metadata, i.e., use validobj to parse metadata_file into CommonMetaData
metadata = parse_yaml_inp(metadata_file, CommonMetaData)

# Now apply variants:
for variant in variants:
    metadata = metadata.apply_variant(variant)
    
# Once we have our final commondata file, parse data, kinematics and uncertainties
data_df = _parse_data(metadata)
uncertainties_df = _parse_uncertainties(metadata)
kin_df = _parse_kinematics(metadata)

ImportError: cannot import name 'CommonMetaData' from 'validphys.commondataparser' (/Users/jumax9/Academic_Workspace/NNPDF/src/nnpdf/validphys2/src/validphys/commondataparser.py)

In [6]:
data_df

,data
entry,
1,0.9815
2,1.0030
3,0.9675
4,1.0330
5,0.9912
...,...
256,0.6717
257,0.7194
258,0.6959


In [7]:
uncertainties_df

name,stat_1,syst_1
treatment,ADD,ADD
type,UNCORR,CORR
entry,,
1,0.0203,0.0109
2,0.0212,0.0134
3,0.0205,0.0112
4,0.0258,0.0195
5,0.0176,0.0121
...,...,...
256,0.0235,0.0034


In [8]:
kin_df

,x,q2,y
,mid,mid,mid
entry,,,
1,0.0015,0.16,0.120852
2,0.0015,0.25,0.188831
3,0.0015,0.35,0.264363
4,0.0015,0.45,0.339895
5,0.0015,0.60,0.453194
...,...,...,...
256,0.6750,26.49,0.044463
257,0.6750,35.40,0.059419


In [9]:
if False:
    # Quickly check that nothing _important_ is broken
    from validphys.api import API
    from validphys.convolution import central_predictions
    pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
    dname = "ATLAS_SINGLETOP_TCH_DIFF_7TEV_T_RAP_NORM" # dataset with apfelcomb flags
    new_ds = API.dataset(dataset_input={"dataset": dname}, theoryid=400, use_cuts="internal")
    print(central_predictions(new_ds, pdf))